In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
yolo=cv.dnn.readNet("D:\programming languages\OpenCV\YOLO\yolov3.weights","D:\programming languages\OpenCV\YOLO\yolov3.cfg")

In [3]:
classes=[]
with open("D:\programming languages\OpenCV\YOLO\coco.names","r") as f:
    classes=f.read().splitlines()

In [4]:
#randomly creating 80 colors of each having 3 channels
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Detecting objects in an Image

In [5]:
def detect_objects_in_image(path):
    img=cv.imread(path)
    blob=cv.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)
    yolo.setInput(blob)
    output_layers_names=yolo.getUnconnectedOutLayersNames()
    outputs=yolo.forward(output_layers_names)
    boxes = []
    confidences = []
    class_ids = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * img.shape[1])
                center_y = int(detection[1] * img.shape[0])
                width = int(detection[2] * img.shape[1])
                height = int(detection[3] * img.shape[0])
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indices = cv.dnn.NMSBoxes(bboxes=boxes, scores=confidences, score_threshold=0.5, nms_threshold=0.4)
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = classes[class_ids[i]]
            confidence = confidences[i]
            color = colors[class_ids[i]]

            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, f"{label}: {confidence:.2f}", (x, y - 10),
                        cv.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv.imshow("Object Detection", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [6]:
detect_objects_in_image(r"D:\programming languages\OpenCV\photoes\person_on_cycle.jpg")

# Detecting objects in a video

In [5]:
def detect_objects_in_video(path=0):
    capture=cv.VideoCapture(path)
    while True:
        isTrue,frame=capture.read()
        if isTrue:
            blob=cv.dnn.blobFromImage(frame,1/255,(416,416),(0,0,0),swapRB=True,crop=False)
            yolo.setInput(blob)
            outputs=yolo.forward(yolo.getUnconnectedOutLayersNames())
            boxes=[]
            confidences=[]
            class_ids=[]
            for output in outputs:
                for prediction in output:
                    scores=prediction[5:]
                    class_id=np.argmax(scores)
                    confidence=scores[class_id]
                    if confidence>0.5:
                        center_x=int(prediction[0]*frame.shape[1])
                        center_y=int(prediction[1]*frame.shape[0])
                        width=int(prediction[2]*frame.shape[1])
                        height=int(prediction[3]*frame.shape[0])
                        x=int(center_x-width/2)
                        y=int(center_y-height/2)
                        boxes.append([x,y,width,height])
                        confidences.append(confidence)
                        class_ids.append(class_id)
            indices=cv.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
            if len(indices)>0:
                for i in indices.flatten():
                    x,y,w,h=boxes[i]
                    cv.rectangle(frame,(x,y),(x+w,y+h),colors[i],2)
                    cv.putText(frame,classes[class_ids[i]]+" : "+str(round(confidences[i],2)),(x,y-10),cv.FONT_HERSHEY_COMPLEX,1.0,colors[i],2)
            cv.imshow("Video",frame)
            if cv.waitKey(1) & 0xFF==ord('q'):
                break
        else:
            break
    capture.release()
    cv.destroyAllWindows()

In [6]:
detect_objects_in_video(r"D:\programming languages\OpenCV\photoes\videos\kitten.mp4")